In [1]:
# header
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

import shared_utils

import pandas as pd
import geopandas as gpd
from siuba import *
from IPython.display import display

pd.set_option('display.max_columns', None) 

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/safety_projects/"

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

# Rank Priority Bridges Per District

In [10]:
def bridge_rank(fn, crashvar):
    with get_fs().open(f'{GCS_FILE_PATH}{fn}.geojson') as f:
        bridges = gpd.read_file(f)
        
    bridges.rename(columns={crashvar:"Crashes"}, inplace=True)
    
    bridges.info()
        
    distcount = (bridges >> count(_.DIST_left))
    print(distcount)
    
    # sort on n crashes, work orders
    bridges = (bridges
               >> arrange(_.DIST_left, -_.Crashes, -_.WO_density)
              )
    # pick top 3 in each district (does not include 15-16th st in Sac)
    bridges_cut = bridges.groupby('DIST_left').apply(lambda x: x.iloc[0:3])
    
    return bridges_cut

## Ped Only

In [11]:
bridges_cut_ped = bridge_rank("priority_bridges_ped_EQI_comparison", "ped_crash")

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   OBJECTID           261 non-null    int64   
 1   OBJECTID1          261 non-null    int64   
 2   OBJECTID_1         261 non-null    int64   
 3   BRIDGE_left        261 non-null    object  
 4   NAME_left          261 non-null    object  
 5   FAC_left           261 non-null    object  
 6   DIST_left          261 non-null    object  
 7   CO_left            261 non-null    object  
 8   NUMBER_KILLED      261 non-null    int64   
 9   NUMBER_INJURED     261 non-null    int64   
 10  Crashes            261 non-null    int64   
 11  bike_crash         261 non-null    int64   
 12  pedbike_crash      261 non-null    int64   
 13  WONO               261 non-null    int64   
 14  WO_density         261 non-null    float64 
 15  WO_dummy           261 non-null    int64   
 16  

In [13]:
# export to csv
(bridges_cut_ped 
 >> mutate(District = _.DIST_left)
 >> filter(_.Crashes>0, _.WONO>0)
 >> select(_.District, _.County==_.CO_left, _.Bridge==_.BRIDGE_left, _.Name==_.NAME_left, _.Facility==_.FAC_left, _.Crashes, _.NUMBER_KILLED, _.NUMBER_INJURED, _.WorkOrders==_.WONO)
).to_csv(f'{GCS_FILE_PATH}ranked_bridgeareas_t3.csv')

In [ ]:
distlist = bridges_cut_ped.DIST_left.unique().tolist()
    
for d in distlist:
    distbridges = (bridges_cut_ped 
                   >> mutate(District = _.DIST_left)
                   >> select(_.District, _.County==_.CO_left, _.BridgeName==_.NAME_left, _.Facility==_.FAC_left, _.Crashes, _.WorkOrders==_.WONO) 
                   >> filter(_.District==d) 
                   >> head(10)
                  )
    display(distbridges)

In [ ]:
bridges_cut_ped.explore()

## Ped and Bike (hold)

In [ ]:
bridges_cut_pedbike = bridge_rank("Priority_Bridges_cut2_EQI", "pedbike_crash")

In [ ]:
distlist = bridges_cut_pedbike.DIST_left.unique().tolist()
    
for d in distlist:
    distbridges = (bridges_cut_pedbike 
                   >> mutate(District = _.DIST_left)
                   >> select(_.District, _.County==_.CO_left, _.BridgeName==_.NAME_left, _.Facility==_.FAC_left, _.Crashes, _.WorkOrders==_.WONO) 
                   >> filter(_.District==d) 
                   >> head(10)
                  )
    display(distbridges)